In [29]:
import pandas as pd
import asyncio
import websockets
import json
import uuid
import time
import requests

In [35]:
async def ping_relay(relay_url):
    rtt_ms = None
    try:
        start = time.time()
        async with websockets.connect(relay_url) as ws:
            end = time.time()
            rtt_ms = int((end - start) * 1000)
    except KeyboardInterrupt:
        raise KeyboardInterrupt
    except Exception as e:
        print(f"Error connecting to {relay_url}: {e}")
    return rtt_ms

def fetch_relay_info(url):
    headers = {'Accept': 'application/nostr+json'}
    data = None
    error = None
    relay_url = url[6:] if url.startswith('wss://') else url[5:] if url.startswith('ws://') else url
    try:
        response = requests.get(f"https://{relay_url}", headers=headers, timeout=10)
        if response.status_code == 200:
            data = response.json()
        else:
            error = response.text
    except Exception:
        try:
            response = requests.get(f"http://{relay_url}", headers=headers, timeout=10)
            if response.status_code == 200:
                data = response.json()
            else:
                error = response.text
        except Exception as e:
            error = str(e)
    return url, {'data': data, 'error': error}

rtt_ms = await ping_relay('wss://relay.damus.io')
url, info = fetch_relay_info('wss://relay.damus.io')

In [37]:
await ping_relay("wss://hcog4c6zjosd3soueyiitcg3jzq6ehwblles5n5snpv477unsrvz26qd.onion")

Error connecting to wss://hcog4c6zjosd3soueyiitcg3jzq6ehwblles5n5snpv477unsrvz26qd.onion: [Errno -2] Name or service not known


In [ ]:
relays = pd.read_csv('../data/raw/relays_url.csv')
for url in relays['url']:
    try:
        async with websockets.connect(url) as ws:
            print(f'Connected to {url}')
    except Exception as e:
        print(f'Failed to connect to {url}: {e}')
    finally:
        print('Done')

Checking wss://relay.damus.io
Connected to wss://relay.damus.io
Done
Checking wss://nostr-pub.wellorder.net
Connected to wss://nostr-pub.wellorder.net
Done
Checking wss://nostr.mom
Connected to wss://nostr.mom
Done
Checking wss://nostr.slothy.win
Failed to connect to wss://nostr.slothy.win: server rejected WebSocket connection: HTTP 403
Done
Checking wss://nostr.einundzwanzig.space
Connected to wss://nostr.einundzwanzig.space
Done
Checking wss://nos.lol
Connected to wss://nos.lol
Done
Checking wss://relay.nostr.band
Connected to wss://relay.nostr.band
Done
Checking wss://no.str.cr
Connected to wss://no.str.cr
Done
Checking wss://relay.cryptocculture.com
Done


CancelledError: 

In [ ]:
RELAY_URL = "wss://relay.nostrdice.com"  # Cambia con l'URL del relay reale
RELAY_URL = "wss://relay.boroghor.com"  # Cambia con l'URL del relay reale
RELAY_URL = "wss://schnorr.me"
RELAY_URL = "wss://mastodon.cloud/api/v1/streaming"
async def fetch_events(from_ts: int, to_ts: int):
    events = []
    subscription_id = str(uuid.uuid4())[:64]
    filter_obj = {
        # "since": from_ts,
        # "until": to_ts,
        # "limit": 10
    }

    async with websockets.connect(RELAY_URL) as ws:
        print(f"Connesso a {RELAY_URL}")

        req_msg = ["REQ", subscription_id, filter_obj]
        await ws.send(json.dumps(req_msg))
        print(f"Inviato: {req_msg}")

        try:
            while True:
                raw_message = await asyncio.wait_for(ws.recv(), timeout=30)
                message = json.loads(raw_message)

                if not isinstance(message, list):
                    print("Messaggio non valido (non è un array JSON)")
                    continue

                msg_type = message[0]

                if msg_type == "EVENT":
                    _, sub_id, event = message
                    if sub_id == subscription_id:
                        # print(f"Ricevuto evento: {event}")
                        events.append(event)

                elif msg_type == "EOSE":
                    _, sub_id = message
                    if sub_id == subscription_id:
                        print("Fine degli eventi storici.")
                        break

                elif msg_type == "NOTICE":
                    _, notice = message
                    print(f"NOTICE dal relay: {notice}")

                elif msg_type == "CLOSED":
                    _, sub_id, reason = message
                    if sub_id == subscription_id:
                        print(f"Subscription chiusa dal relay: {reason}")
                        break

                elif msg_type == "OK":
                    _, event_id, accepted, message_str = message
                    status = "accettato" if accepted else "rifiutato"
                    print(f"Evento {event_id} {status} - {message_str}")

                else:
                    print(f"Messaggio sconosciuto: {message}")

        except asyncio.TimeoutError:
            print("Timeout: nessun messaggio ricevuto per 30 secondi.")
        finally:
            close_msg = ["CLOSE", subscription_id]
            await ws.send(json.dumps(close_msg))
            print("Subscription chiusa.")
    return events

# Esegui in una cella async
async def main():
    to_time = int(time.time())
    from_time = to_time - 600
    return await fetch_events(0, 1727090175)

events = await main()


InvalidStatus: server rejected WebSocket connection: HTTP 403

In [22]:
def generate_nostr_keypair():
    """
    Generate a Nostr-compatible secp256k1 keypair.

    Returns:
    - dict: {
        'private_key': str,   # 64-char hex
        'public_key': str,    # 66-char hex (compressed)
        'nsec': str,          # Bech32 encoded private key
        'npub': str           # Bech32 encoded public key
      }
    """
    import subprocess
    import bech32
    import secp256k1

    def convert_hex_to_bech32(hex_str: str, prefix: str) -> str:
        byte_data = bytes.fromhex(hex_str)
        data = bech32.convertbits(byte_data, 8, 5, True)
        return bech32.bech32_encode(prefix, data)

    # 1. Generate private key (32 bytes → 64 hex chars)
    result = subprocess.run(
        ["openssl", "rand", "-hex", "32"],
        capture_output=True,
        text=True,
        check=True
    )
    private_key = result.stdout.strip()

    # 2. Derive compressed public key (33 bytes → 66 hex chars)
    priv_key_bytes = bytes.fromhex(private_key)
    privkey_obj = secp256k1.PrivateKey(privkey=priv_key_bytes, raw=True)
    public_key = privkey_obj.pubkey.serialize(compressed=True).hex()

    # 3. Bech32 encode
    nsec = convert_hex_to_bech32(private_key, "nsec")
    npub = convert_hex_to_bech32(public_key, "npub")

    return {
        "private_key": private_key,
        "public_key": public_key,
        "nsec": nsec,
        "npub": npub
    }

generate_nostr_keypair()

{'private_key': 'c531802c3fb1fdd5e1bef1e20c9016a5eea0227617ad938be2a1328a45d33702',
 'public_key': '0312cf93bdbcdcb9e9f4ff20692a295f55d5140e7b1316b0430b30966aef9ab5b0',
 'nsec': 'nsec1c5ccqtplk87atcd7783qeyqk5hh2qgnkz7ke8zlz5yeg53wnxupqja70qz',
 'npub': 'npub1qvfvlyaahnwtn605lusxj23fta2a29qw0vf3dvzrpvcfv6h0n26mqp03hwh'}

In [1]:
import utils

sec, pub = utils.generate_nostr_keypair()
nsec = utils.to_bech32('nsec', sec)
npub = utils.to_bech32('npub', pub)
print(f"Private Key: {sec}")
print(f"Public Key: {pub}")
print(f"Bech32 Private Key: {nsec}")
print(f"Bech32 Public Key: {npub}")

event = utils.generate_event(sec, pub, 1, 1, [], '')
utils.verify_sig(event['id'], event['pubkey'], event['sig'])

Private Key: f06e01066e4fd74eb09df7b2c0b32991135e061d8321c24d20f59a8e146e0702
Public Key: fae8b85a8ad4db03af84a41af4e86f959805f02a0eb8e652f4e6eb7c83ddb435
Bech32 Private Key: nsec17phqzpnwflt5avya77evpvefjyf4upsasvsuynfq7kdgu9rwqupqzh7zu8
Bech32 Public Key: npub1lt5tsk526nds8tuy5sd0f6r0jkvqtup2p6uwv5h5um4heq7aks6s6fx9yk


ValueError: The public key could not be parsed or is invalid.

In [ ]:
from coincurve import PrivateKey, PublicKey


def generate_nostr_keypair():
    """
    Generates a Nostr-style keypair.

    Returns:
    - Tuple[str, str]: (private_key_hex, pubkey_hex) where pubkey is 32-byte x-only.
    """
    sk = PrivateKey()
    pk = sk.public_key.format(compressed=True)[1:]  # Drop prefix for x-only format
    return sk.to_hex(), pk.hex()


def sign_schnorr(event_id: str, private_key_hex: str) -> str:
    """
    Sign a 32-byte message with Schnorr using coincurve (BIP340).

    Returns:
    - str: Hex-encoded Schnorr signature.
    """
    if not isinstance(event_id, str) or not isinstance(private_key_hex, str):
        raise TypeError("Arguments must be hex strings.")

    private_key = PrivateKey(bytes.fromhex(private_key_hex))
    signature = private_key.sign_schnorr(bytes.fromhex(event_id))
    return signature.hex()


def verify_schnorr(event_id: str, pubkey_hex: str, sig_hex: str) -> bool:
    """
    Verify a Schnorr signature using coincurve.

    Parameters:
    - pubkey_hex (str): 32-byte x-only public key (as in Nostr)
    - sig_hex (str): Hex-encoded Schnorr signature
    """
    if not all(isinstance(arg, str) for arg in (event_id, pubkey_hex, sig_hex)):
        raise TypeError("All arguments must be hex strings.")

    try:
        # Convert x-only pubkey to full compressed key (default prefix 0x02)
        full_pubkey = b'\x02' + bytes.fromhex(pubkey_hex)
        pubkey = PublicKey(full_pubkey)
        return pubkey.schnorr_verify(bytes.fromhex(event_id), bytes.fromhex(sig_hex))
    except Exception:
        return False



# Generate keypair
priv, pub = generate_nostr_keypair()
print("Private:", priv)
print("Public:", pub)

# Simulate a message (e.g., an event_id hash)
message = "e41d2f51b631d627f1c5ed83d66e1535ac0f1542a94db987c93f758c364a7600"

# Sign
signature = sign_schnorr(message, priv)
print("Signature:", signature)

# Verify
is_valid = verify_schnorr(message, pub, signature)
print("Valid signature?", is_valid)


Private: 6176c182e78a3c47ab84dfcf5949a6471122bc751ecb5785e5140d94d3873428
Public: fbec510b831f92b7566193a9d53e480bf41cb1755c336abc9917df3cc5946015
Signature: 9c4657abbeb21c0559355a59bcaf2c5aca53c03dc6cc727b213f6dda2f6776615f48e5143d2b213097a9b8660c4024a3aece8b358917f82a9e827c50fe505b79
Valid signature? False
